In [175]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import requests
import re
import validators
from googletrans import Translator
from translate import Translator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from sklearn.preprocessing import StandardScaler

In [92]:
data = pd.read_csv("./Dataset - EMPOLITICON NLP and ML Based Approach for Context and Emotion Classification of Political Speeches From Transcripts.csv")
data.head()

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,Russia,16/07/2021,Vladimir Putin,Meeting of APEC Economic Leaders,"Madam Chair,\n\nColleagues,\n\nFirst of all, ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,OPTIMISM,DEVELOPMENT
1,Russia,2021-09-05 00:00:00,Vladimir Putin,Victory Parade on Red Square,"Citizens of Russia,\n\nDear veterans,\n\nComra...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,JOY,NATIONALISM
2,Russia,2021-08-04 00:00:00,Vladimir Putin,Meeting on the results of implementing Preside...,"Good afternoon, colleagues.\n\nLet’s start.\n\...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,DEVELOPMENT
3,Russia,21-11-2020,Vladimir Putin,G20 Summit,"Colleagues,\n\nThe scope of problems humanity ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,DEVELOPMENT
4,Russia,20-11-2020,Vladimir Putin,Address to participants in Nuremberg Lessons f...,"Colleagues, friends,\n\nFirst of all, I would ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,UPSET,EXTREMISM


In [189]:
x_train = data["Text_of_Speech"]
y_train = data["Emotion"]

In [190]:
null_value_index = x_train.loc[x_train.isnull()].index
print(null_value_index)

Int64Index([], dtype='int64')


In [191]:
x_train.drop(null_value_index, axis=0,inplace = True)
y_train.drop(null_value_index, axis=0,inplace = True)

In [192]:
null_value_index = y_train.loc[y_train.isnull()].index
print(null_value_index)

Int64Index([], dtype='int64')


In [193]:
x_train.drop(null_value_index, axis=0,inplace = True)
y_train.drop(null_value_index, axis=0,inplace = True)

In [194]:
y_train.isnull().sum()

0

In [195]:
stop_words = stopwords.words("english")

In [196]:
def remove_un(text):
    te = ""
    for i in text.split("\n"):
        if len(i) > 0:
            te = te + i 
    return te

In [197]:
def remove_un1(text):
    te = ""
    for i in text.split("\t"):
        if len(i) > 0:
            te = te + i 
    return te

In [198]:
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

In [199]:
x_train = x_train.apply(remove_un)
x_train = x_train.apply(remove_un1)
x_train = x_train.apply(remove_numbers)

In [200]:
lem = WordNetLemmatizer()

In [201]:
def removing_stop_words(text):
    data = nltk.word_tokenize(text)
    te = ""
    for i in data:
        if i.strip().lower() not in stop_words:
            new = lem.lemmatize(i)
            te = te + new + " "
    return te

In [202]:
x_train = x_train.apply(removing_stop_words)

In [143]:
x_train,x_test,y_train,y_test = train_test_split(x_train,y_train,test_size = 0.25,random_state = 42)

In [171]:
x_train_list = speeches = x_train.tolist()
bag_of_words = [nltk.word_tokenize(speech.lower()) for speech in x_train_list]

In [179]:
x_test_list = speeches = x_test.tolist()
bag_of_words2 = [nltk.word_tokenize(speech.lower()) for speech in x_test_list]

In [91]:
word2vec_model = Word2Vec(sentences=bag_of_words, vector_size=100, window=5, min_count=1, sg=0)
word2vec_model.train(bag_of_words, total_examples=len(bag_of_words), epochs=10)
vector = word2vec_model.wv['word']

In [181]:
word2vec_model2 = Word2Vec(sentences=bag_of_words2, vector_size=100, window=5, min_count=1, sg=0)
word2vec_model.train(bag_of_words2, total_examples=len(bag_of_words2), epochs=10)
vector = word2vec_model.wv['word']

In [166]:
def aggregate_vectors(sentence, model):
    vector_sum = np.zeros(model.vector_size)
    num_words = 0
    for word in sentence:
        if word in model.wv:
            vector_sum += model.wv[word]
            num_words += 1
    if num_words > 0:
        return vector_sum / num_words
    else:
        return np.zeros(model.vector_size)

In [172]:
X_train_word2vec = [aggregate_vectors(sentence, word2vec_model) for sentence in bag_of_words ]
X_train_word2vec_matrix = np.vstack(X_train_word2vec)

In [182]:
X_train_word2vec = [aggregate_vectors(sentence, word2vec_model2) for sentence in bag_of_words2]
X_test_word2vec_matrix = np.vstack(X_train_word2vec)

In [173]:
print(len(X_train_word2vec_matrix))

1507


In [203]:
vectorizer = TfidfVectorizer()
X_train_count_matrix=vectorizer.fit_transform(x_train)
# X_test_count_matrix = vectorizer.transform(x_test)

#### logistic regression

In [204]:
logic = LogisticRegression()
logic.fit(X_train_count_matrix,y_train)
print(logic.score(X_train_count_matrix,y_train))
# y_pred = logic.predict(X_test_count_matrix)
# print("accuracy = ",accuracy_score(y_test,y_pred))

0.8189054726368159


#### using the word to vector modeel

In [188]:
logic = LogisticRegression(max_iter=1000)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_word2vec_matrix)

logic.fit(X_train_scaled ,y_train)
# print(logic.score(X_train_count_matrix,y_train))
y_pred = logic.predict(X_test_word2vec_matrix)
print("accuracy = ",accuracy_score(y_test,y_pred))

accuracy =  0.2962226640159046


#### SVC

In [148]:
svc = SVC(probability=True)
svc.fit(X_train_count_matrix,y_train)
# print(logic.score(X_train_count_matrix,y_train))
y_pred = svc.predict(X_test_count_matrix)
print("accuracy = ",accuracy_score(y_test,y_pred))

accuracy =  0.48111332007952284


#### decision Tree

In [163]:
dec = DecisionTreeClassifier()
dec.fit(X_train_count_matrix,y_train)
y_pred = dec.predict(X_test_count_matrix)
print("accuracy = ",accuracy_score(y_test,y_pred))

accuracy =  0.3518886679920477


#### random forest

In [152]:
ran = RandomForestClassifier()
ran.fit(X_train_count_matrix,y_train)
y_pred = ran.predict(X_test_count_matrix)
print("accuracy = ",accuracy_score(y_test,y_pred))

accuracy =  0.48906560636182905


#### k nearest nighbours

In [165]:
knn = KNeighborsClassifier()
knn.fit(X_train_count_matrix,y_train)
y_pred = knn.predict(X_test_count_matrix)
print("accuracy = ",accuracy_score(y_test,y_pred))

accuracy =  0.4234592445328032


In [159]:
final = VotingClassifier(estimators=[('logic', LogisticRegression()), ('SVC',SVC(probability=True)),("ran",RandomForestClassifier())], voting='soft').fit(X_train_count_matrix, y_train)

In [160]:
ypred4 = final.predict(X_test_count_matrix)

In [161]:
print("accuracy = ",accuracy_score(y_test,ypred4))

accuracy =  0.48906560636182905
